In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')
import vectorbtpro as vbt
import os, sys, io, time, math, pickle, copy, json, asyncio, random, zipfile, functools, itertools
import datetime as dt
import talib.abstract as taa
import talib as ta
import pandas as pd
import numpy as np

In [3]:
from dataloader import Dataloader
from backtest import Backtest, CommonParams, Direction
import indicators as indi

dataloader = Dataloader()
backtest = Backtest()

### Type symbol and period

In [4]:
symbol = 'BTCUSDT'
start_date = '01.2024' # строго в формате MM.YYYY
end_date = '06.2024' # строго в формате MM.YYYY

### Load data from binance.futures
save .csv files in "klines/{symbol}".
⚠️Skeep if data has been loaded before

In [5]:
# dataloader.download(symbol, start_date, end_date)

### Read data
1m candle stick data for:
1. Open trades based on indicator signals
2. Trade simulation

In [6]:
df = dataloader.read(symbol, start_date, end_date).reset_index(drop=True)

### Signal

In [ ]:
# Preparing data for signal
timeframe = '2h'

signal_df = dataloader.convert_tf(df, timeframe)
ohlcv = {c: signal_df[c].values for c in signal_df.columns}

In [8]:
# def ma(source, length, ma_type):
#     if ma_type == "SMA":
#         return source.rolling(window=length).mean()
#     elif ma_type == "Bollinger Bands":
#         return source.rolling(window=length).mean()
#     elif ma_type == "EMA":
#         return source.ewm(span=length, adjust=False).mean()
#     elif ma_type == "SMMA (RMA)":
#         smma = [source[0]]
#         for i in range(1, len(source)):
#             smma.append((smma[i-1] * (length - 1) + source[i]) / length)
#         return pd.Series(smma, index=source.index)
#     elif ma_type == "WMA":
#         weights = np.arange(1, length + 1)
#         return source.rolling(window=length).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)
#     return None

# def calculate_rsi_and_ma(prices):
#     rsiLengthInput = 14
#     rsiSourceInput = prices
#     maTypeInput = "SMA"
#     maLengthInput = 14
#     bbMultInput = 2.0

#     change = rsiSourceInput.diff()
#     up = change.where(change > 0, 0)
#     down = -change.where(change < 0, 0)
#     up = ma(up, rsiLengthInput, "SMMA (RMA)")
#     down = ma(down, rsiLengthInput, "SMMA (RMA)")

#     rsi = 100 - (100 / (1 + up / down))

#     rsiMA = ma(rsi, maLengthInput, maTypeInput)
#     rsiMA = np.where(np.isnan(rsiMA), -1, rsiMA)
#     rsi = np.where(np.isnan(rsi), -1, rsi)

#     return {"rsi": rsi, "rsi_ma": rsiMA}

# data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
# data['close'] = data['close'].astype(float)
# prices = data['close'] # open, high, close, low
# rsi_data = calculate_rsi_and_ma(prices)
# rsi = rsi_data['rsi']
# rsi_ma = rsi_data['rsi_ma']
# correction = rsi - rsi_ma
# result = np.where((correction >= 25) | (correction <= -25), 1, 0) # Тут коррекцию менять

# cur = np.nan_to_num(ta.STOCHRSI(ohlcv['open'], timeperiod=14, fastk_period=20, fastd_period=3, fastd_matype=3))
# cur = np.nan_to_num(indi.stochrsi(ohlcv['open'], rsi_period=14, rsimin_period=20, rsimax_period=20, d_period=1)*100)
cur = np.nan_to_num(indi.stochrsi(ohlcv['open'], rsi_period=6, rsimin_period=8, rsimax_period=8, d_period=1)*100)
prev = np.roll(cur, 1)
prev[0] = 0
result = np.where((cur>20) & (prev<20), 1, 0)
# result = np.not_equal(np.prod(np.where((cur>50) & (prev<50), 1, 0), axis=0), 0)

signal_df['signal'] = result
print(print('signal count: ', signal_df['signal'].sum()))

signal count:  145
None


In [11]:
# renko = indi.renko(ohlcv['close'], box_size=0.01)
# renko

In [12]:
# long = np.where(renko>np.roll(renko, 1), 1, 0)
# short = np.where(renko<np.roll(renko, 1), 1, 0)
# short[0] = 0
# print(np.sum(long))
# print(np.sum(short))

In [13]:
# unique_prices = renko[np.insert(np.diff(renko) != 0, 0, True)]
# pd.DataFrame(unique_prices).plot()

### Preparing data for backtest

In [9]:
bdf = df.merge(signal_df[['datetime', 'signal']], on='datetime', how='left').fillna(0) # signal mapping with main df

In [10]:
step = .01 # шаг перебора параметров выхода
# take_profits=np.append(np.nan, np.arange(.30, step))                # take profit %
take_profits=[np.nan]                                                     # take profit %
# stop_losses=np.append(np.nan, np.arange(.05, step))                       # stop loss %
stop_losses=[np.nan, .05, .10, .12]                       # stop loss %
trailing_stop_activation=[np.nan, 0.025, .05]    # trailing stop activation % - на каком процентном росте активируется трейлинг стоп
trailing_stops=np.append(np.nan, np.arange(0.03, 0.08, step*2))         # traling stop % - если np.nan то не используется
# формируем параметры
common_params = CommonParams(
    open_date_time=bdf['datetime'].values,
    open=bdf['open'].values,
    high=bdf['high'].values,
    low=bdf['low'].values,
    close=bdf['close'].values,
    entries=bdf['signal'].values,                                             # точки входа
    take_profits=take_profits,                                                # take profit %
    stop_losses=stop_losses,                                                  # stop loss %
    trailing_stop_activation=trailing_stop_activation,                        # trailing stop activation % - на каком процентном росте активируется трейлинг стоп
    trailing_stops=trailing_stops,                                            # traling stop % - если np.nan то не используется
    init_cash=10000,                                                           # начальный баланс $
    size=1000,                                                                 # сумма входа в сделку $
    fees=.0004,                                                               # комиссия $
    slippage=.0000,                                                           # проскальзывание %
    direction=Direction.LONG                                                  # направление
    # direction=Direction.SHORT                                                 # направление
)

In [ ]:
# run backtest
cpu_usage_weight = .08
group_size = 30 
result = backtest.run(common_params, cpu_usage_weight, group_size)

### Save backtest results

In [12]:
filename = f'{symbol} {timeframe} {start_date}-{end_date} d1.xlsx'
backtest.save(filename, result)